# Ensemble Spread and Statistical Significance
### Authors

Samantha Stevenson sstevenson@ucsb.edu

### Table of Contents

[Goals](#purpose)

[Import Packages](#path)

[Load and Query the CMIP6 AWS Catalog](#load)

[Pull Data of Interest: Historical Plus SSP](#data_io)

[Apply CA State Region Mask](#mask)

[Build PDFs of Regionally Averaged Data](#pdfs)

[Do Gridpoint Significance Testing](#sigtest)

<a id='purpose'></a> 
## **Goals**

In this tutorial, we will learn some techniques for assessing _statistical significance_ of changes in climate data, and ways to represent significance on map and time series plots!

This will allow you to practice skills learned in previous tutorials:
- [Mapping Climate Data](https://github.com/climate-datalab/Map-Plots/blob/main/1.%20Mapping%20Climate%20Data.ipynb) (putting spatial data onto a map using Cartopy)
- [Plotting Regional Time Series Using Shapefiles](https://github.com/climate-datalab/EnsembleAnalysis/blob/main/2.%20Plotting%20Regional%20Time%20Series%20Using%20Shapefiles.ipynb)  (masking out irregular regions from the climate model grid using shape files)

while also learning some new skills that will be presented below:
- **Database querying strategies** to take a more detailed look at what's inside the CMIP catalog;
- **Creating probability distribution functions** based on either time-varying information from a _single_ ensemble member, or time-averaged information from _multiple_ ensemble members;
- **Applying T-tests and rank-sum tests** to compute the level of statistical significance of differences; and
- **Stippling map areas** to show where differences are and are not significant!

<a id='path'></a> 
## **Import Packages**

As always, we begin by importing the necessary packages for our analysis. 

1) Packages from previous tutorial

This tutorial assumes you have all the packages from tutorial 2 ([Plotting Regional Time Series Using Shapefiles](https://github.com/climate-datalab/EnsembleAnalysis/blob/main/2.%20Plotting%20Regional%20Time%20Series%20Using%20Shapefiles.ipynb)) installed; if you need more information on these packages, please refer to that tutorial.

2) New package: `scipy`

In addition to these packages, we'll also be working with a new one called Scipy! This is a really powerful package that's designed to do all sorts of complex scientific computations using the features of Numpy - some examples include solving differential equations and interpolating between missing data points. Here we're mainly going to be using Scipy's statistics functionality, which is stored in the `scipy.stats` sub-package. We can just load that one into our environment as `stats`.

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import intake
import s3fs
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.geometry import Point
import scipy.stats as stats

<a id='load'></a> 
## **Load and Query the CMIP6 AWS Catalog**

As we did in previous tutorials, we'll load the CMIP6 database hosted by Amazon Web Services:

In [2]:
# Open the CMIP6 data catalog, store as a variable
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')

To make things a bit more interesting, let's practice looking through the catalog, to find a couple of good model ensembles to work with. 

Let's say we're trying to find some ensembles of **historical** output! Here is a recommended workflow to look through the catalog and pick some data.

1) Specify the `experiment_id`: since the "historical" experiments are ALL part of the "CMIP" activity, you DON'T need to also set the `activity_id` field here.

2) Specify the `table_id` and `variable_id` fields: sometimes different sets of variables are available with different models, so picking out one variable and time frequency can be helpful in identifying the exact ensemble members you'll be using.

**NOTE:** up till now, all the tutorials relied on surface air temperature ("tas") data. Let's try something else! Precipitation is called `pr` in CMIP jargon, and is an atmosphere variable like surface air temperature. We'll stick with monthly data to make the file sizes manageable, which means we'll still be using the `Amon` "table_id"!

In [4]:
# Specify search terms to query catalog 
# experiment_id: what experimental configuration do you want? Here we want historical 
experiment_ids = ['historical']
# table_id: which part of the Earth system and time resolution do you want? Here we want monthly atmosphere data
table_id = 'Amon' 
# variable_id: which climate variable do you want? 
variable_id = 'pr'

# Extract the part of the catalog that goes with these search terms
res = catalog.search(experiment_id=experiment_ids, table_id=table_id, variable_id=variable_id)

2) Take a look at the data frame to see how many different models there are that ran historical simulations: you can do this by applying the `.unique()` function to the `source_id` field in the catalog dataframe.

In [5]:
# Find all unique models that ran historical simulations
print(res.df.source_id.unique())

['GFDL-CM4' 'GFDL-ESM4' 'IPSL-CM6A-LR' 'GISS-E2-1-G' 'CNRM-CM6-1'
 'BCC-CSM2-MR' 'CNRM-ESM2-1' 'MIROC6' 'BCC-ESM1' 'MRI-ESM2-0'
 'SAM0-UNICON' 'CESM2' 'GISS-E2-1-H' 'UKESM1-0-LL' 'CESM2-WACCM' 'CanESM5'
 'CanESM5-CanOE' 'INM-CM4-8' 'INM-CM5-0' 'HadGEM3-GC31-LL'
 'MPI-ESM-1-2-HAM' 'NESM3' 'CAMS-CSM1-0' 'MPI-ESM1-2-LR' 'MPI-ESM1-2-HR'
 'E3SM-1-0' 'MCM-UA-1-0' 'NorESM2-LM' 'GISS-E2-1-G-CC' 'FGOALS-g3'
 'MIROC-ES2L' 'KACE-1-0-G' 'NorCPM1' 'FGOALS-f3-L' 'CNRM-CM6-1-HR'
 'NorESM2-MM' 'ACCESS-CM2' 'ACCESS-ESM1-5' 'GISS-E2-2-H' 'CESM2-WACCM-FV2'
 'CESM2-FV2' 'HadGEM3-GC31-MM' 'FIO-ESM-2-0' 'E3SM-1-1' 'IITM-ESM'
 'EC-Earth3-Veg' 'EC-Earth3' 'AWI-ESM-1-1-LR' 'EC-Earth3-Veg-LR'
 'AWI-CM-1-1-MR' 'CMCC-CM2-SR5' 'E3SM-1-1-ECA' 'TaiESM1'
 'EC-Earth3-AerChem' 'IPSL-CM5A2-INCA' 'CMCC-CM2-HR4' 'CAS-ESM2-0'
 'EC-Earth3-CC' 'CMCC-ESM2' 'MIROC-ES2H' 'ICON-ESM-LR' 'IPSL-CM6A-LR-INCA'
 'KIOST-ESM']


3) Choose a couple of entries from the above at random, and see how many ensemble members are available in each. For the activities below we're going to want to have a fair amount of data available: let's set 5 members as an arbitrary minimum threshold. Let's also not use HUGE ensembles here, to make the code run faster! So... **can we find models that have between 5 and 10 historical ensemble members??**

To answer this question, pick some model names, enter them as the `source_id` term in the call to `catalog.search`, then from the data frame that comes back as a result display the unique values of `member_id`. You can even get fancy, and **loop** over all the possible model `source_ids` then display the unique set of ensemble members that goes with each one!

This is demonstrated in the code block below:

In [7]:
# source_id: which model do you want? 
source_id = res.df.source_id.unique()

for src in source_id:
    # Print out the name of the model we're working with, to keep track of things
    print(src)
    
    # Extract the part of the catalog that goes with this model
    this_res = catalog.search(experiment_id=experiment_ids, source_id=src, table_id=table_id, variable_id=variable_id)

    # Print the unique member ids
    print(this_res.df.member_id.unique())

GFDL-CM4
['r1i1p1f1']
GFDL-ESM4
['r3i1p1f1' 'r2i1p1f1' 'r1i1p1f1']
IPSL-CM6A-LR
['r4i1p1f1' 'r6i1p1f1' 'r5i1p1f1' 'r9i1p1f1' 'r15i1p1f1' 'r18i1p1f1'
 'r16i1p1f1' 'r29i1p1f1' 'r30i1p1f1' 'r28i1p1f1' 'r3i1p1f1' 'r31i1p1f1'
 'r7i1p1f1' 'r8i1p1f1' 'r2i1p1f1' 'r27i1p1f1' 'r26i1p1f1' 'r21i1p1f1'
 'r25i1p1f1' 'r20i1p1f1' 'r19i1p1f1' 'r22i1p1f1' 'r1i1p1f1' 'r12i1p1f1'
 'r10i1p1f1' 'r17i1p1f1' 'r11i1p1f1' 'r24i1p1f1' 'r23i1p1f1' 'r14i1p1f1'
 'r13i1p1f1' 'r32i1p1f1']
GISS-E2-1-G
['r2i1p1f1' 'r1i1p1f1' 'r5i1p1f1' 'r4i1p1f1' 'r3i1p1f1' 'r6i1p1f1'
 'r7i1p1f1' 'r9i1p1f1' 'r8i1p1f1' 'r10i1p1f1' 'r2i1p3f1' 'r5i1p3f1'
 'r6i1p3f1' 'r9i1p3f1' 'r3i1p3f1' 'r10i1p3f1' 'r8i1p3f1' 'r4i1p3f1'
 'r1i1p3f1' 'r102i1p1f1' 'r101i1p1f1' 'r4i1p1f2' 'r1i1p1f2' 'r11i1p1f2'
 'r1i1p1f3' 'r5i1p1f2' 'r6i1p1f2' 'r5i1p1f3' 'r2i1p1f2' 'r10i1p1f2'
 'r3i1p1f2' 'r7i1p1f2' 'r2i1p1f3' 'r9i1p1f2' 'r3i1p1f3' 'r4i1p1f3'
 'r8i1p1f2' 'r1i1p5f1' 'r6i1p5f1' 'r7i1p5f1' 'r10i1p5f1' 'r2i1p5f1'
 'r8i1p5f1' 'r9i1p5f1' 'r4i1p5f1' 'r3i1p5f1']
CN

After some playing with the results, I've decided to go with the following:
- **ACCESS-CM2: 5 ensemble members**

You can change this around if you want though!

For a bit more information about ACCESS-CM2: this is an Australian climate model, the Australian Community Climate and Earth System Simulator. They have lots of background information about ACCESS on the [ACCESS homepage](https://research.csiro.au/access/about/)!

**helpful tip:** if you're curious about any given model you might come across, just Google the name! There's probably a handy webpage out there waiting to explain it to you.

<a id='data_io'></a> 
## **Pull Data of Interest: Historical Plus SSP**

Now that we've picked a model, let's go ahead and collect all the data for it. 

As an additional task, I'm ALSO going to demonstrate how to extract information for BOTH the historical and future projection data associated with a given model ensemble, and concatenate these together in a way that matches ensemble members to one another. This is a tool that will come in handy throughout your climate modeling adventures, since it allows you to:
- be sure that you don't have any weird discontinuities in your time series from mismatches between ensemble members
- easily subset your data for any time period you want, without worrying about whether it's in the historical or the SSP dataset

We'll begin with more catalog queries, for our selected model ACCESS-CM2. But now we're going to do it twice: once for the historical simulations and once for the future projections! This means choosing a future scenario - just for the sake of picking something, we'll choose SSP3-7.0 (or in the CMIP6 catalog, `ssp370`).

In [13]:
# Extract data just for ACCESS-CM2
# Specify search terms to query catalog 
source_id='ACCESS-CM2'

# Extract historical data
res_access_hist = catalog.search(experiment_id='historical', source_id=source_id, table_id=table_id, 
                                 variable_id=variable_id)
# Extract future SSP projection data
res_access_ssp = catalog.search(experiment_id='ssp370', source_id=source_id, table_id=table_id, 
                                 variable_id=variable_id)

In [16]:
# Print the data frames
print(res_access_hist.df)

print(res_access_ssp.df)

  activity_id institution_id   source_id experiment_id member_id table_id  \
0        CMIP   CSIRO-ARCCSS  ACCESS-CM2    historical  r1i1p1f1     Amon   
1        CMIP   CSIRO-ARCCSS  ACCESS-CM2    historical  r2i1p1f1     Amon   
2        CMIP   CSIRO-ARCCSS  ACCESS-CM2    historical  r3i1p1f1     Amon   
3        CMIP   CSIRO-ARCCSS  ACCESS-CM2    historical  r4i1p1f1     Amon   
4        CMIP   CSIRO-ARCCSS  ACCESS-CM2    historical  r5i1p1f1     Amon   

  variable_id grid_label                                             zstore  \
0          pr         gn  s3://cmip6-pds/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-...   
1          pr         gn  s3://cmip6-pds/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-...   
2          pr         gn  s3://cmip6-pds/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-...   
3          pr         gn  s3://cmip6-pds/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-...   
4          pr         gn  s3://cmip6-pds/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-...   

   dcpp_init_year   version  
0             NaN  20191108  
1 

From printing out the data frames above and looking at the `member_id` field, we can see that this dataset is nice and well behaved: all the same ensemble members are available in the historical and the SSP3-7.0 ensembles, and the members are listed in the same order. **This will not always be the case!!** Sometimes modeling centers don't extend all historical simulations out to 2100 using the SSP scenarios, and sometimes they'll end up reordered in random ways relative to one another. 

What do we do about this situation? It's not too bad, really: we just have to make sure that as we're reading in data, the ensemble members _match_ between the historical and future projection simulations.

We'll go through essentially the same process outlined in tutorials 1 and 2 in this repo: looping over each unique ensemble member, reading in the data from the zarr store as an individual xarray Dataset, then building a list of xarray objects. From there, we'll use `xr.concat` to transform the list into a larger xarray Dataset with a new dimension called `member`, the coordinate for which is the name of each unique ensemble member.

**But now there's a twist**: the members we'll loop over will be ONLY those that are in BOTH the historical AND the future projection ensembles!

Not only that, but we'll also add code that will automatically LOCATE the ensemble member we want inside the two data frames we created above. 

#### **Here's how it works:**

First, we locate the members that are common across the historical and SSP ensembles. We can do this by converting the lists of unique members from each ensembles to sets using the `set()` command, then calculating the [_set intersection_](https://www.w3schools.com/python/ref_set_intersection.asp) between them (syntax for this is `set1 & set2`), then converting this back to a list using the `list()` command. To make the code shorter, I did all of the above in a single line:

In [ ]:
# Make a list of the unique ensemble members
# historical
mems_hist = res_access_hist.df.member_id.unique()
# SSP
mems_ssp = res_access_ssp.df.member_id.unique()

# Convert these to sets, find the intersection between them, and convert back to a list
common_mems = list(set(mems_hist) & set(mems_ssp))

Now we've got a new list of members to iterate over! In this particular case, it's the exact same list we had before - but again, that likely won't be the case in every example you encounter.

We apply the same approach of looping over the number of unique ensemble members - but now, there's new line code _inside_ the loop, that does a couple of things:
- finds where the particular ensemble member we want is in each data frame (historical and SSP)
- extracts data individually from each data frame from that location
- concatenates the historical and SSP data across the time dimension

THEN puts the concatenated data for that ensemble member into the list of output data!

In [28]:
# Define an empty list for output data
access_data = []

# Retrieve number of members the historical and SSP ensembles have in common
num = len(common_mems)

# Loop over all members-in-common
for mem in range(num):
    # Print statement to keep track of which member we're working on
    print(common_mems[mem])
    
    # Figure out where this member is in the historical ensemble
    # True/False array showing whether or not the member_id matches our member of interest
    hist_mask = res_access_hist.df['member_id'] == common_mems[mem]
    # extract (first) location where the mask is True
    hist_loc = res_access_hist.df['member_id'][hist_mask].index[0]
    
    # Do the same thing for the SSP ensemble
    ssp_mask = res_access_ssp.df['member_id'] == common_mems[mem]
    ssp_loc = res_access_ssp.df['member_id'][ssp_mask].index[0]
    
    # Extract data from each entry as xarray
    temp_data_hist = xr.open_zarr(res_access_hist.df['zstore'][hist_loc], storage_options={'anon': True})
    temp_data_ssp = xr.open_zarr(res_access_ssp.df['zstore'][ssp_loc], storage_options={'anon': True})
    
    # Concatenate the historical and SSP data across the time dimension
    temp_data = xr.concat([temp_data_hist, temp_data_ssp], dim="time")
    
    # Add the concatenated data to a list
    access_data.append(temp_data)

# Concatenate the list into a single xarray object
access_data = xr.concat(access_data, dim="member")

# Store the actual member information as values of the new dimension
access_data = access_data.assign_coords(member=("member", common_mems))

r1i1p1f1
r2i1p1f1
r4i1p1f1
r5i1p1f1
r3i1p1f1


In [29]:
# Print out the full dataset to make sure it looks right
access_data

<xarray.Dataset>
Dimensions:    (lat: 144, bnds: 2, lon: 192, member: 5, time: 3012)
Coordinates:
  * lat        (lat) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
    lat_bnds   (lat, bnds) float64 -90.0 -88.75 -88.75 ... 88.75 88.75 90.0
  * lon        (lon) float64 0.9375 2.812 4.688 6.562 ... 355.3 357.2 359.1
    lon_bnds   (lon, bnds) float64 0.0 1.875 1.875 3.75 ... 358.1 358.1 360.0
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2100-12-16T12:00:00
    time_bnds  (time, bnds) datetime64[ns] 1850-01-01 1850-02-01 ... 2101-01-01
  * member     (member) <U8 'r1i1p1f1' 'r2i1p1f1' ... 'r5i1p1f1' 'r3i1p1f1'
Dimensions without coordinates: bnds
Data variables:
    pr         (member, time, lat, lon) float32 dask.array<chunksize=(1, 495, 144, 192), meta=np.ndarray>
Attributes: (12/50)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    cmor_version:           3.4.0
    ...                     ...
    tracking_id:            hdl:21.14100/b4dd0f13-6073-4d10-b4e6-7d7a4401e37d
    variable_id:            pr
    variant_label:          r1i1p1f1
    version:                v20191108
    netcdf_tracking_ids:    hdl:21.14100/b4dd0f13-6073-4d10-b4e6-7d7a4401e37d
    version_id:             v20191108

The print statement above should show that the output data has dimension:
- lat and lon (should correspond to the same sizes as each original input data set)
- member (should be the same length as the `common_mems` list we built above)
- time (should be the sum of the lengths of the historical and SSP datasets - or 12 x the number of total years from 1850-2100)
- bnds (this is the weird dimension of length 2, that relates to the boundaries between time steps or lat/lon grids - you don't need this one)

<a id='mask'></a> 
## **Apply CA State Region Mask**

Now that we've determined what variable and set of historical model ensembles we're interested in, let's go further and pull out the data that covers just the state of California! We covered the basic steps involved with this in [tutorial 2](https://github.com/climate-datalab/EnsembleAnalysis/blob/main/2.%20Plotting%20Regional%20Time%20Series%20Using%20Shapefiles.ipynb). 

First, read in the shape file for the CA state boundary and reproject it to a projection of your choice (here I am using PlateCarree since it's what we were working with in earlier mapping tutorials):

In [30]:
# Read in shapefile for CA state boundary
gdf = gpd.read_file('ca_state/CA_State.shp')

# Reproject the shapefile to use the PlateCarree projection
gdf = gdf.to_crs(epsg=4326)

Next, determine which lat/lon points in the model grid you're working with fall within the boundaries of that shape file. We've been doing this using the following procedure:
1) Convert the model lat and lon coordinates to two-dimensional variables if they are currently 1D

2) Convert the longitude values to the -180 to 180 convention if they currently range from 0 to 360

3) Create a set of Shapely "points" from the set of all (lat,lon) combinations in the model grid

4) Combine these points into a GeoDataFrame using the same coordinate reference system as the shape file

5) Perform a spatial join of the shape file and the model grid GeoDataFrame

Code to perform these steps is below:

In [31]:
# Make 2D lat, lon
lon_vals = access_data.lon.values
# Convert values of longitude greater than 180 to negative values
lon_vals = np.where(lon_vals > 180, lon_vals - 360, lon_vals)
lon2d, lat2d = np.meshgrid(lon_vals, access_data.lat.values)

# Build a list of 'points' from the model grid
points = [Point(lon, lat) for lon, lat in zip(lon2d.flatten(), lat2d.flatten())]

# Create a GeoDataFrame from the xarray dataset's coordinates
points_gdf = gpd.GeoDataFrame(geometry=points, crs="EPSG:4326")

# Spatial join to find points within the shapefile
joined = gpd.sjoin(points_gdf, gdf)

After the process of identifying the points inside and outside of the shape file, you still have to apply this as a mask to the actual climate data!

We did **this** using the following procedure:

1) Build an array of indices having the same length as the climate model grid

2) Mask out the indices that are outside the shape file (stored in the `joined` variable above)

3) Reshape the array to have the same _shape_ as the climate model grid

4) Apply the logical mask to the actual climate variable

Code to perform these steps is below:

In [32]:
# Identify indices of "good" data
# total number of points
num_points = points_gdf.shape[0]
# make an array of indices of length num_points
inds_array = np.arange(num_points)

# Make a logical mask that tells you whether or not 
# that index is in the set of joined points
mask = np.isin(inds_array, joined.index)

# Reshape to the original shape of the lat/lon grid
mask_2d = mask.reshape(lat2d.shape)

# Apply mask to the data
masked_data = access_data.where(mask_2d)

<a id='pdfs'></a> 
## **Build PDFs of Regionally Averaged Data**

Now we're finally at the point of being able to do some analysis! 

We'll first use the mask from the previous section to create regionally averaged time series for the state of California, then start building some probability distribution functions (PDFs). 

#### Why would you want to do this??

Knowing how to build PDFs is a really useful skill! This is an important way that scientists and environmental managers determine the significance of changes between different time periods: or in other words, it's how you can tell that there's something different going on that you might need to plan for.

This can come in handy, for instance, when you're trying to figure out whether the likelihood of extremely damaging events like heat waves or extreme rainstorms has changed. Or maybe you're just trying to see if there really is ANY effect of climate change on your particular variable of interest!


In [ ]:
kde_int1_clust0 = stats.gaussian_kde(regint1_clust0)
